# OpenAI: Exercises

Large Language Models are highly dependant on the Prompt. This includes the wording of the context, the description of the categories, examples provided, and the question asked, as well as the formatting of the prompt.

This is called prompt engineering, and it can be, at times, an arcane art. Prompts that work well on one model may not work on another. Seemingly arbitrary changes such as linebreaks may influence the model output. This is due to the probabilistic nature of large languae models.

Usually, prompt engineering is a matter of trial and error more than anything else. There is a plethora of (quickly changing) articles on the matter, but here are some ressources to get you started:

https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-the-openai-api

https://medium.com/mantisnlp/prompt-engineering-part-ii-how-to-construct-prompts-de333dcf4c85

https://huggingface.co/docs/transformers/tasks/prompting#best-practices-of-llm-prompting


## Zero-Shot Classification

Load the same UK media data as before. Try altering the system prompt to get better results. 

What happens when you alter the context - e.g. telling the model that it is very good at its job? What happens when you alter the question? Also try changing the labels provided to the model. Take a look at the [CAP codebook](https://www.comparativeagendas.net/pages/master-codebook) - can you come up with better descriptions? Also try giving the categories more descriptive names than just a number (you might have to change the max_tokens returned by the model in this case to allow for longer output!).

You can also choose a different model to run your tests and see if another GPT model is faring better. See the [OpenAI Documentation](https://platform.openai.com/docs/models#models-overview) for an overview of models.

Hint: A smaller dataset will help speed up the process. It also allows you to inspect the results more thoroughly, to get a feel for the effects of your prompt engineering. When you feel like you came up with a good prompt, try running it on the same sample as before and see if results improved!

In [ ]:
# we're loading up our trusty UK media dataset and do some minor data cleaning

import pandas as pd

seed = 20250228

uk_media = pd.read_csv('data/uk_media.csv')

 # fillna() makes sure missing values don't result in NaN entries
uk_media['text'] = uk_media['description'].fillna('') + ' ' + uk_media['subtitle'].fillna('')

# we'll also drop duplicates indicated by the filter_duplicate column
uk_media = uk_media[uk_media['filter_duplicate'] == 0]

# we'll also drop rows where text is NaN (missing due to missing headlines)
uk_media = uk_media[uk_media['text'].notna()]

# drop rows with majortopic code 0
uk_media = uk_media[uk_media['majortopic'] != 0]

# only keep rows below 24 OR equal to 99
uk_media = uk_media[(uk_media['majortopic'] < 24) | (uk_media['majortopic'] == 99)]

# drop category 22 not in the CAP
uk_media = uk_media[uk_media['majortopic'] != 22]

# turn the majortopic column into a string
uk_media['majortopic'] = uk_media['majortopic'].astype(str)

# a small sample for prompt engineering
uk_media_sample_sm = uk_media.sample(n=10, random_state=seed)
uk_media_sample_sm.reset_index(drop = True, inplace = True) # reset index


In [ ]:
# our classification function

import re

def classify_text(text, system_message, model, max_tokens, logprobs, top_logprobs):

  # clean the text by removing extra spaces
  text = re.sub(r'\s+', ' ', text).strip()

  # construct input

  messages = [
    # system prompt
    {"role": "system", "content": system_message}, # this will contain all instructions for the model
    # user input
    {"role": "user", "content": text}, # text here is the input text to be classified
  ]

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0,
    max_tokens=max_tokens, # restrict max tokens for more consistent/concise output
    logprobs=logprobs, # include log probs
    top_logprobs=top_logprobs, # number of top logprobs to return
  )

  
  result = response.choices[0].message.content

  if logprobs: # a simple check if we wanted logprobs to get returned
    logprobs = response.choices[0].logprobs.content[0].logprob
    top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
    return result, logprobs, top_logprobs
  
  else: # if not, return only the result
    return result

In [1]:
# Edit the prompt here

# the CAP labels
cap_labels = {
    "1": "Issues related to general domestic macroeconomic policy; Interest Rates; Unemployment Rate; Monetary Policy; National Budget; Tax Code; Industrial Policy; Price Control; other macroeconomics subtopics",
    "2": "Issues related generally to civil rights and minority rights; Minority Discrimination; Gender Discrimination; Age Discrimination; Handicap Discrimination; Voting Rights; Freedom of Speech; Right to Privacy; Anti-Government; other civil rights subtopics",
    "3": "Issues related generally to health care, including appropriations for general health care government agencies; Health Care Reform; Insurance; Drug Industry; Medical Facilities; Insurance Providers; Medical Liability; Manpower; Disease Prevention; Infants and Children; Mental Health; Long-term Care; Drug Coverage and Cost; Tobacco Abuse; Drug and Alcohol Abuse; health care research and development; issues related to other health care topics",
    "4": "Issues related to general agriculture policy, including appropriations for general agriculture government agencies; agricultural foreign trade; Subsidies to Farmers; Food Inspection & Safety; Food Marketing & Promotion; Animal and Crop Disease; Fisheries & Fishing; agricultural research and development; issues related to other agricultural subtopics",
    "5": "Issues generally related to labor, employment, and pensions, including appropriations for government agencies regulating labor policy; Worker Safety; Employment Training; Employee Benefits; Labor Unions; Fair Labor Standards; Youth Employment; Migrant and Seasonal workers; Issues related to other labor policy",
    "6": "Issues related to General education policy, including appropriations for government agencies regulating education policy; Higher education, student loans and education finance, and the regulation of colleges and universities; Elementary & Secondary education; Underprivileged students; Vocational education; Special education and education for the physically or mentally handicapped; Education Excellence; research and development in education; issues related to other subtopics in education policy",
    "7": "Issues related to General environmental policy, including appropriations for government agencies regulating environmental policy; Drinking Water; Waste Disposal; Hazardous Waste; Air Pollution; Recycling; Indoor Hazards; Species & Forest; Land and Water Conservation; research and development in environmental technology, not including alternative energy; issues related to other environmental subtopics",
    "8": "Issues generally related to energy policy, including appropriations for government agencies regulating energy policy; Nuclear energy, safety and security, and disposal of nuclear waste; Electricity; Natural Gas & Oil; Coal; Alternative & Renewable Energy; Issues related to energy conservation and energy efficiency; issues related to energy research and development; issues related to other energy subtopics",
    "9": "Issues related to immigration, refugees, and citizenship",
    "10": "Issues related generally to transportation, including appropriations for government agencies regulating transportation policy; mass transportation construction, regulation, safety, and availability; public highway construction, maintenance, and safety; Air Travel; Railroad Travel; Maritime transportation; Infrastructure and public works, including employment initiatives; transportation research and development; issues related to other transportation subtopics",
    "12": "Issues related to general law, crime, and family issues; law enforcement agencies, including border, customs, and other specialized enforcement agencies and their appropriations; White Collar Crime; Illegal Drugs; Court Administration; Prisons; Juvenile Crime; Child Abuse; Family Issues, domestic violence, child welfare, family law; Criminal & Civil Code; Crime Control; Police; issues related to other law, crime, and family subtopics",
    "13": "Issues generally related to social welfare policy; Low-Income Assistance; Elderly Assistance; Disabled Assistance; Volunteer Associations; Child Care; issues related to other social welfare policy subtopics",
    "14": "Issues related generally to housing and urban affairs; housing and community development, neighborhood development, and national housing policy; urban development and general urban issues; Rural Housing; economic, infrastructure, and other developments in non-urban areas; housing for low-income individuals and families, including public housing projects and housing affordability programs; housing for military veterans and their families, including subsidies for veterans; housing for the elderly, including housing facilities for the handicapped elderly; housing for the homeless and efforts to reduce homelessness ; housing and community development research and development; Other issues related to housing and community development",
    "15": "Issues generally related to domestic commerce, including appropriations for government agencies regulating domestic commerce; Banking; Securities & Commodities; Consumer Finance; Insurance Regulation; personal, commercial, and municipal bankruptcies; corporate mergers, antitrust regulation, corporate accounting and governance, and corporate management; Small Businesses; Copyrights and Patents; Disaster Relief; Tourism; Consumer Safety; Sports Regulation; domestic commerce research and development; other domestic commerce policy subtopics",
    "16": "Issues related generally to defense policy, and appropriations for agencies that oversee general defense policy; defense alliance and agreement, security assistance, and UN peacekeeping activities; military intelligence, espionage, and covert operations; military readiness, coordination of armed services air support and sealift capabilities, and national stockpiles of strategic materials.; Nuclear Arms; Military Aid; military manpower, military personel and their dependents, military courts, and general veterans' issues; military procurement, conversion of old equipment, and weapons systems evaluation; military installations, construction, and land transfers; military reserves and reserve affairs; military nuclear and hazardous waste disposal and military environmental compliance; domestic civil defense, national security responses to terrorism, and other issues related to homeland security; non-contractor civilian personnel, civilian employment in the defense industry, and military base closings; military contractors and contracting, oversight of military contrators and fraud by military contractors; Foreign Operations; claims against the military, settlements for military dependents, and compensation for civilians injured in military operations; defense research and development; other defense policy subtopics",
    "17": "Issues related to general space, science, technology, and communications; government use of space and space resource exploitation agreements, government space programs and space exploration, military use of space; regulation and promotion of commercial use of space, commercial satellite technology, and government efforts to encourage commercial space development; science and technology transfer and international science cooperation; Telecommunications; Broadcast; Weather Forecasting; computer industry, regulation of the internet, and cyber security; space, science, technology, and communication research and development not mentioned in other subtopics.; other issues related to space, science, technology, and communication research and development",
    "18": "Issues generally related to foreign trade and appropriations for government agencies generally regulating foreign trade; Trade Agreements; Exports; Private Investments; productivity of competitiveness of domestic businesses and balance of payments issues; Tariff & Imports; Exchange Rates; other foreign trade policy subtopics",
    "19": "Issues related to general international affairs and foreign aid, including appropriations for general government foreign affairs agencies; Foreign Aid; Resources Exploitation; Developing Countries; International Finance; Western Europe; issues related specifically to a foreign country or region not codable using other codes, assessment of political issues in other countries, relations between individual countries; Human Rights; International organizations, NGOs, the United Nations, International Red Cross, UNESCO, International Olympic Committee, International Criminal Court; international terrorism, hijacking, and acts of piracy in other countries, efforts to fight international terrorism, international legal mechanisms to combat terrorism; diplomats, diplomacy, embassies, citizens abroad, foreign diplomats in the country, visas and passports; issues related to other international affairs policy subtopics",
    "20": "Issues related to general government operations, including appropriations for multiple government agencies; Intergovernmental Relations; Bureaucracy; Postal Service; issues related to civil employees not mentioned in other subtopics, government pensions and general civil service issues; issues related to nominations and appointments not mentioned elsewhere; issues related the currency, national mints, medals, and commemorative coins; government procurement, government contractors, contractor and procurement fraud, and procurement processes and systems; government property management, construction, and regulation; Tax Administration; public scandal and impeachment; government branch relations, administrative issues, and constitutional reforms; regulation of political campaigns, campaign finance, political advertising and voter registration; Census & Statistics; issues related to the capital city; claims against the government, compensation for the victims of terrorist attacks, compensation policies without other substantive provisions; National Holidays; other government operations subtopics",
    "21": "Issues related to general public lands, water management, and territorial issues; National Parks; Indigenous Affairs; natural resources, public lands, and forest management, including forest fires, livestock grazing; water resources, water resource development and civil works, flood control, and research; territorial and dependency issues and devolution; other public lands policy subtopics",
    "23": "Issues related to general cultural policy issues",
    "99": "Other issues, where none of the above is appropriate.", # dummy category
}

# give the model some context for the task it is about to perform
context = """
You are a political scientist tasked with annotating documents into policy categories. 
The documents can be classified as one of the following numbered categories. 
A description of each category is following the ':' sign.
"""

# turn the CAP dictionary into a string
labels_definitions = ""

for i in range(len(cap_labels)):
    labels_definitions += f'{list(cap_labels.keys())[i]}: {list(cap_labels.values())[i]}\n'

# finally, the question we want the model to answer, including specific instructions for the output
question = """
Which policy category does this document belong to? 
Answer only with the number of the category, and only with a single category.
"""

# now we combine the parts into the system prompt
system_message = f"{context}\n{labels_definitions}\n\n{question}"

print(system_message)
print(f'Prompt length: {len(system_message)}')


You are a political scientist tasked with annotating documents into policy categories. 
The documents can be classified as one of the following numbered categories. 
A description of each category is following the ':' sign.

1: Issues related to general domestic macroeconomic policy; Interest Rates; Unemployment Rate; Monetary Policy; National Budget; Tax Code; Industrial Policy; Price Control; other macroeconomics subtopics
2: Issues related generally to civil rights and minority rights; Minority Discrimination; Gender Discrimination; Age Discrimination; Handicap Discrimination; Voting Rights; Freedom of Speech; Right to Privacy; Anti-Government; other civil rights subtopics
3: Issues related generally to health care, including appropriations for general health care government agencies; Health Care Reform; Insurance; Drug Industry; Medical Facilities; Insurance Providers; Medical Liability; Manpower; Disease Prevention; Infants and Children; Mental Health; Long-term Care; Drug Covera

In [ ]:
# test it on the small sample

model = "gpt-4o-mini-2024-07-18"

classification_results = [classify_text(text, 
                                        system_message = system_message, 
                                        model = model, 
                                        logprobs = True,
                                        top_logprobs=5, # number of alternative logprobs
                                        max_tokens = 10) for text in uk_media_sample_sm['text']] # we're looping our function over the texts

classification_results_df = pd.concat([uk_media_sample_sm, 
                                       pd.DataFrame(classification_results, 
                                                    columns = ['result', 'logprobs', 'top_logprobs'])],
                                        axis = 1)
classification_results_df

When you feel like your prompt engineering has paid off, run your prompt on the same sample as before and evaluate the results. Did the outcome improve?

In [ ]:
# set a seed before sampling for repdroducibility
seed = 20250228 # I'm using today's date as seed here

uk_media_sample = uk_media.sample(n = 100, random_state = seed) # we'll sample 100 out of the 17746 rows

uk_media_sample.reset_index(drop = True, inplace = True) # reset index

# ...

### Solution

In [ ]:
classification_results = [classify_text(text, 
                                        system_message = system_message, 
                                        model = model, 
                                        logprobs = True,
                                        top_logprobs=5, # number of alternative logprobs
                                        max_tokens = 10) for text in uk_media_sample['text']] # we're looping our function over the texts

classification_results_df = pd.concat([uk_media_sample, 
                                       pd.DataFrame(classification_results, 
                                                    columns = ['result', 'logprobs', 'top_logprobs'])],
                                        axis = 1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(classification_results_df["majortopic"], classification_results_df["result"]))

## Few-Shot Classification

Try adding examples for few-shot classification. Does it help to add more or less examples? Does the formatting of the examples improve the outcome? Do different models fare better with or without examples?

Hint: Do not forget to make a train/test split before pulling examples. You can use the same function as before.

In [ ]:
from sklearn.model_selection import train_test_split

# this functions pulls n samples per category from the dataframe as train, leaving the rest as test
def stratified_train_test_split(df, category_col, n_train_per_category):
    train_dfs = []
    test_dfs = []
    
    for category, group in df.groupby(category_col):
        train_group, test_group = train_test_split(group, train_size=n_train_per_category, random_state=42)
        train_dfs.append(train_group)
        test_dfs.append(test_group)
    
    train_df = pd.concat(train_dfs).reset_index(drop=True)
    test_df = pd.concat(test_dfs).reset_index(drop=True)
    
    return train_df, test_df


train_sample, test_sample = stratified_train_test_split(uk_media, 'majortopic', 2) # we'll use 2 samples per category. Edit this as needed

In [ ]:
uk_media_sample_fewshot_sm = test_sample.sample(n = 10, random_state = seed)

uk_media_sample_fewshot_sm.reset_index(drop = True, inplace = True) # reset index

In [ ]:
# Edit the prompt here

# give the model some context for the task it is about to perform
context = """
You are a political scientist tasked with annotating documents into policy categories. 
The documents can be classified as one of the following numbered categories. 
A description of each category is following the ':' sign.
You will be provided two examples for each category to help you make a decision. These are marked with "Examples:".
"""

# turn the CAP dictionary into a string
labels_definitions = ""

for label in cap_labels.keys():
    examples = train_sample[train_sample['majortopic'] == label]['text'].values
    labels_definitions += f'{label}:\n{cap_labels[label]}\nExamples: {examples}\n'

# finally, the question we want the model to answer, including specific instructions for the output
question = """
Which policy category does this document belong to? 
Answer only with the number of the category, and only with a single category.
"""

# now we combine the parts into the system prompt
system_message = f"{context}\n{labels_definitions}\n\n{question}"

print(system_message)

In [ ]:
# ...

### Solution

In [ ]:
# set a seed before sampling for repdroducibility
seed = 20250228 # I'm using today's date as seed here

uk_media_sample_fewshot_sm = test_sample.sample(n = 100, random_state = seed)

uk_media_sample_fewshot_sm.reset_index(drop = True, inplace = True) # reset index

classification_results_fewshot = [classify_text(text, 
                                        system_message = system_message, 
                                        model = model, 
                                        logprobs = True,
                                        top_logprobs=5, # number of alternative logprobs
                                        max_tokens = 10) for text in uk_media_sample_fewshot_sm['text']] # we're looping our function over the texts

classification_results_fewshot_df = pd.concat([uk_media_sample_fewshot_sm, 
                                       pd.DataFrame(classification_results_fewshot, 
                                                    columns = ['result', 'logprobs', 'top_logprobs'])],
                                        axis = 1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(classification_results_fewshot_df["majortopic"], classification_results_fewshot_df["result"]))

## Bonus: Other Datasets

If you have other datasets you would like to classify, you can try feeding them into the model and evaluate results. Come up with appropriate prompts, and remember that you need some kind of gold standard to properly evaluate results.

If you do not have your own data, you can try classifying the CAP data on German party manifestos found under `data/manifesto_1949-2013_website-release_2.5_2_3.csv`. Does the English prompt work well for German data? The GPT models are multilingual, but does it change results if you use a German prompt instead?